<center><h1>Covid 19 - Indian Districts at a Glance</h1></center>

<p>Hover over state to update charts.</p>

In [1]:
import pandas as pd
import numpy as np
import bqplot as bqp
from bqplot.market_map import MarketMap
from ipywidgets import Label, HBox, VBox, Layout
from bqplot.traits import convert_to_date

In [2]:
url = 'https://api.covid19india.org/csv/latest/districts.csv'
data = pd.read_csv(url)

In [3]:
cdata = data
cdata = cdata.drop(columns=["Other","Tested"])

In [4]:
idx = cdata[(cdata["District"]=="Unknown") | (cdata["District"]=="Others") | 
            (cdata["District"]=="State Pool") |
           (cdata["District"]=="Foreign Evacuees") |
           (cdata["District"]=="Other State") |
           (cdata["District"]=="Italians") |
           (cdata["District"]=="Airport Quarantine") |
           (cdata["District"]=="Railway Quarantine") |
           (cdata["District"]=="Evacuees")].index
cdata.drop(idx, inplace = True)

In [5]:
    popdata = pd.read_csv('distpop.csv')
    sdf = pd.DataFrame(columns = ["state","district","total_cases","total_deaths","growth_rate","cpm","dpm",'prev',"population","censuscode"])
    udf = cdata.groupby(["State","District"]).size().reset_index()
    
    for i in range(len(udf)):
        df = cdata[(cdata["State"]==udf.iloc[i]["State"]) & (cdata["District"]==udf.iloc[i]["District"])]
        state = df["State"].iloc[0]
        district = df["District"].iloc[0]
        total_deaths = df["Deceased"].max()
        total_cases = df["Confirmed"].max()
            
        p =  popdata[(popdata["State"]==state) & (popdata["District"]==district)]
        if not p.empty:
            pop = p["Population"].iloc[0]
            censuscode = p['Censuscode'].iloc[0].astype(int)
            
        dpm = round(total_deaths*1e6/pop,2)
        cpm = round(total_cases*1e6/pop,2)
        
        d = df["Deceased"]
        delta = d.diff()
        gr = delta[1::]*100/d.iloc[0:len(d)]
        gr = gr[-14::]
        gr_mean = round(gr.mean(),2)
        prev = round(dpm*0.0666,2)
        
    
        
        sdf = sdf.append({'state': state, 'district': district, 'total_cases': total_cases,'total_deaths': total_deaths,
                         'growth_rate': gr_mean, 'cpm': cpm, "dpm": dpm, 'prev': prev, "population": pop, "censuscode": censuscode},
                        ignore_index=True)
        
    idx = sdf[sdf["growth_rate"]<0].index
    sdf.drop(idx, inplace = True)
    idx = sdf[sdf["total_deaths"]==0].index
    sdf.drop(idx, inplace = True)
    sdf = sdf.replace([np.inf, -np.inf],np.nan)
    sdf.dropna()
    sdf['censuscode'] = sdf['censuscode'].astype(int, errors='ignore')

In [6]:
# Cases Timeseries

sc_x = bqp.DateScale()
sc_y = bqp.LinearScale()

ax_x = bqp.Axis(scale=sc_x, grid_lines='dashed', label='Date', num_ticks = 4, label_location='end',tick_format='%b %y')
ax_y = bqp.Axis(scale=sc_y, orientation='vertical', grid_lines='dashed',
            label='Confirmed Cases', label_location='end', tick_format='.2s')

line = bqp.Lines(x=[], y=[], scales={'x': sc_x, 'y': sc_y}, colors=['orange'])
fig_tooltip = bqp.Figure(marks=[line], axes=[ax_x, ax_y], layout=Layout(width='600px', height='400px'), animation_duration=1000)

In [7]:
# Cases Heatmap

col = bqp.ColorScale(scheme='Reds')
ax_c = bqp.ColorAxis(scale=col, label='Cases per million')
p = bqp.market_map.MarketMap(names=sdf['district'], display_text= [''],    
           ref_data=sdf, 
           scales={'color': col}, axes=[ax_c],
           colors = ['#ccc'], tooltip_fields=['state','district','cpm'],
           groups = sdf['state'], cols=25, row_groups=3,
           title = 'Covid-19 - Indian Districts - Cases per million',
           color = sdf['cpm'],
           layout=Layout(width='600px', height='400px'))

In [8]:
# Update the tooltip chart
hovered_symbol = ''
def hover_handler(self, content):

    global hovered_symbol
    symbol = content.get('ref_data', {}).get('district', '')+', '+content.get('ref_data', {}).get('state', '')
    if(symbol != hovered_symbol):
        st = content.get('ref_data', {}).get('state', '')
        dt = content.get('ref_data', {}).get('district', '')

        line.x = convert_to_date(data[(data['District']==dt) & (data['State']==st)]['Date'].values)
        line.y = data[(data['District']==dt) & (data['State']==st)]['Confirmed'].values
        fig_tooltip.title = content.get('ref_data', {}).get('district', '')+', '+content.get('ref_data', {}).get('state', '')
        line2.x = convert_to_date(data[(data['District']==dt) & (data['State']==st)]['Date'].values)
        line2.y = data[(data['District']==dt) & (data['State']==st)]['Deceased'].values
        fig_tooltip2.title = content.get('ref_data', {}).get('district', '')+', '+content.get('ref_data', {}).get('state', '')

p.on_hover(hover_handler)

In [9]:

# Creating the figure to be displayed as the tooltip

sc_x2 = bqp.DateScale()
sc_y2 = bqp.LinearScale()

ax_x2 = bqp.Axis(scale=sc_x2, grid_lines='dashed', label='Date', num_ticks = 4, label_location='end',tick_format='%b %y')
ax_y2 = bqp.Axis(scale=sc_y2, orientation='vertical', grid_lines='dashed',
            label='Deaths', label_location='end', tick_format='.2s')

line2 = bqp.Lines(x=[], y=[], scales={'x': sc_x2, 'y': sc_y2}, colors=['orange'])
fig_tooltip2 = bqp.Figure(marks=[line2], axes=[ax_x2, ax_y2], layout=Layout(width='600px', height='400px'), animation_duration=1000)

In [10]:
# Deaths Heatmap

col = bqp.ColorScale(scheme='Reds')
ax_c2 = bqp.ColorAxis(scale=col, label='Deaths per million')
p2 = bqp.market_map.MarketMap(names=sdf['district'], display_text= [''],    
           ref_data=sdf, 
           scales={'color': col}, axes=[ax_c2],
           colors = ['#ccc'], tooltip_fields=['state','district','dpm'],
           groups = sdf['state'], cols=25, row_groups=3,
           title = 'Covid-19 - Indian Districts - Deaths per million',
           color = sdf['dpm'],
           layout=Layout(width='600px', height='400px'))

In [11]:
# Update the tooltip chart
# hovered_symbol = ''
# def hover_handler(self, content):

#     global hovered_symbol
#     symbol = content.get('data', '')
#     if(symbol != hovered_symbol):
#         hovered_symbol = symbol

#     if(symbol != hovered_symbol):
#         st = content.get('ref_data', {}).get('state', '')
#         dt = content.get('ref_data', {}).get('district', '')

#         line.x = convert_to_date(data[(data['District']==dt) & (data['State']==st)]['Date'].values)
#         line.y = data[(data['District']==dt) & (data['State']==st)]['Confirmed'].values
#         fig_tooltip.title = content.get('ref_data', {}).get('district', '')+', '+content.get('ref_data', {}).get('state', '')
#         line2.x = convert_to_date(data[(data['District']==dt) & (data['State']==st)]['Date'].values)
#         line2.y = data[(data['District']==dt) & (data['State']==st)]['Deceased'].values
#         fig_tooltip2.title = content.get('ref_data', {}).get('district', '')+', '+content.get('ref_data', {}).get('state', '')

p2.on_hover(hover_handler)

In [12]:
row1 = HBox([p,fig_tooltip])
row2 = HBox([p2,fig_tooltip2])
VBox([row1,row2])

In [13]:
c = cdata[data['District']=='Bilaspur']
c

<ipython-input-13-526e42e51fcb>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = cdata[data['District']=='Bilaspur']


,Date,State,District,Confirmed,Recovered,Deceased
38,2020-04-26,Chhattisgarh,Bilaspur,1,1,0
449,2020-04-27,Chhattisgarh,Bilaspur,1,1,0
873,2020-04-28,Chhattisgarh,Bilaspur,1,1,0
1303,2020-04-29,Chhattisgarh,Bilaspur,1,1,0
1737,2020-04-30,Chhattisgarh,Bilaspur,1,1,0
...,...,...,...,...,...,...
237127,2021-05-01,Himachal Pradesh,Bilaspur,5556,4279,33
237719,2021-05-02,Chhattisgarh,Bilaspur,53781,45529,791
237781,2021-05-02,Himachal Pradesh,Bilaspur,5769,4398,33
238373,2021-05-03,Chhattisgarh,Bilaspur,54795,46392,806
